# Orbit Propagation Validation Plotting for XYZ seperate components

This tutorial demonstrates how to perform orbit propagation using the propagated orbit mode.

## Setup

In [ ]:
import numpy as np
import pandas as pd
import csv
import os
import sys

import plotly.graph_objs as go

from ostk.mathematics.geometry.d3.objects import Cuboid
from ostk.mathematics.geometry.d3.objects import Composite
from ostk.mathematics.geometry.d3.objects import Point

from ostk.physics.units import Mass
from ostk.physics.units import Length
from ostk.physics.units import Derived
from ostk.physics.units import Angle
from ostk.physics.units import Time
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Velocity
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth
from ostk.physics.environment.objects.celestial_bodies import Sun
from ostk.physics.environment.objects.celestial_bodies import Moon

from ostk.astrodynamics import NumericalSolver
from ostk.astrodynamics.trajectory import State
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.flight.system import SatelliteSystem
from ostk.astrodynamics.flight.system.dynamics import SatelliteDynamics
from ostk.astrodynamics.trajectory.orbit.models import Propagated

---

## Propagated Orbit Model

Read in reference data from CSV file 

In [ ]:
filename = 'GMAT_TwoBodyGravity_24hr_30sInterval_run.csv'
plot_title_extension = 'comparison with GMAT EGM96 Gravity Model'

multiplication_array = np.array([1.0e3, 1.0e3, 1.0e3])

start_instant = Instant.date_time(DateTime.parse("2021-03-20T00:00:00.000"), Scale.UTC)
frame_GCRF = Frame.GCRF()

with open(f'{os.getcwd()}/Outputfiles/{filename}', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    reference_state_array = [] 
    instant_array = []
    for row in reader:
        
        instant_iter = start_instant + Duration.seconds(float(row['YAM.ElapsedSecs']))
        position_iter = Position.meters([row['YAM.EarthICRF.X'], row['YAM.EarthICRF.Y'], row['YAM.EarthICRF.Z']], frame_GCRF)
        velocity_iter = Velocity.meters_per_second([row['YAM.EarthICRF.VX'], row['YAM.EarthICRF.VY'], row['YAM.EarthICRF.VZ']], frame_GCRF)
        
        
        position_iter_m = np.multiply(position_iter.get_coordinates(), multiplication_array)
        velocity_iter_ms = np.multiply(velocity_iter.get_coordinates(), multiplication_array)

        
        instant_array.append(instant_iter)
        reference_state_array.append(State(instant_iter, Position.meters(position_iter_m, frame_GCRF), Velocity.meters_per_second(velocity_iter_ms, frame_GCRF)))

### Computation

Create an environment corresponding to the perturbations that should be used

In [ ]:
instant_J2000 = Instant.J2000()
objects = [Earth.default()]

custom_env = Environment(instant_J2000, objects)

Create a satellite system, a satellite dynamical system, and a numerical solver

In [ ]:
mass = Mass(90.0, Mass.Unit.Kilogram)
satellite_geometry = Composite(Cuboid(Point(0.0, 0.0, 0.0), [ [1.0, 0.0, 0.0 ], [ 0.0, 1.0, 0.0 ], [ 0.0, 0.0, 1.0 ] ], [1.0, 0.0, 0.0 ] ))
inertia_tensor = np.ndarray(shape=(3, 3))
surface_area = 0.8
drag_coefficient = 2.2

satellitesystem = SatelliteSystem(mass, satellite_geometry, inertia_tensor, surface_area, drag_coefficient)

start_state = reference_state_array[0]

satellitedynamics = SatelliteDynamics(custom_env, satellitesystem, start_state)

numericalsolver = NumericalSolver(NumericalSolver.LogType.NoLog, NumericalSolver.StepperType.RungeKuttaCashKarp54, 5.0, 1.0e-15, 1.0e-15)

Setup a Propagated model an an instant array at which propagated states are desired

In [ ]:
propagated_model = Propagated(satellitedynamics, numericalsolver)

Setup the orbit (TBI) to be able to use the QSW reference frame afterwords 

In [ ]:
orbit = Orbit(propagated_model, custom_env.access_celestial_object_with_name('Earth'))
frame_QSW = orbit.get_orbital_frame(Orbit.FrameType.QSW)

Now that everything is set up, we can calculate the state arrays from the desired time instant grid

In [ ]:
propagated_state_array = orbit.get_states_at(instant_array)

In [ ]:
def to_dataframe (propagated_state):

    return [
                repr(propagated_state.get_instant()),
                float((propagated_state.get_instant() - propagated_state_array[0].get_instant()).in_seconds()),
                *propagated_state.get_position().get_coordinates(),
                *propagated_state.get_velocity().get_coordinates(),
            ]

In [ ]:
propagated_orbit_data = [to_dataframe(propagated_state) for propagated_state in propagated_state_array]

In [ ]:
propagated_orbit_df = pd.DataFrame(propagated_orbit_data, columns=['$Time^{UTC}$', 'Elapsed secs', '$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$', '$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$'])

Table:

In [ ]:
propagated_orbit_df.head()

In [ ]:
propagated_orbit_df.tail()

#### Trajectory delta vs GMAT run in the GCRF frame 

In [ ]:
def to_dataframe_GCRF_diff (ind):
    
    return [
                repr(propagated_state_array[ind].get_instant()),
                
                float((propagated_state_array[ind].get_instant() - propagated_state_array[0].get_instant()).in_seconds()),

                *(propagated_state_array[ind].get_position().get_coordinates() - reference_state_array[ind].get_position().get_coordinates()),
                
                # print(f'Prop:{(propagated_state_array[ind].in_frame(frame_QSW).get_position().get_coordinates())}'),
                # print(f'Ref:{(reference_state_array[ind].in_frame(frame_QSW).get_position().get_coordinates())}'),
                *(propagated_state_array[ind].get_velocity().get_coordinates() - reference_state_array[ind].get_velocity().get_coordinates()),
            ]

In [ ]:
orbit_data_diff_GCRF = [to_dataframe_GCRF_diff(ind) for ind in range(0,len(propagated_state_array))]

# orbit_data_diff_QSW = [to_dataframe_QSW_diff(ind) for ind in range(0,2)]

In [ ]:
orbit_df_diff_GCRF = pd.DataFrame(orbit_data_diff_GCRF, columns=['$Time^{UTC}$', 'Elapsed secs', '${\delta}x_{x}^{ECI}$', '${\delta}x_{y}^{ECI}$', '${\delta}x_{z}^{ECI}$', '${\delta}v_{x}^{ECI}$', '${\delta}v_{y}^{ECI}$', '${\delta}v_{z}^{ECI}$'])

In [ ]:
orbit_df_diff_GCRF.head()

In [ ]:
orbit_df_diff_GCRF.tail()

#### Trajectory delta vs GMAT run in the RTN frame 

In [ ]:
def to_dataframe_QSW_diff (ind):
    
    return [
                repr(propagated_state_array[ind].get_instant()),
                float((propagated_state_array[ind].get_instant() - propagated_state_array[0].get_instant()).in_seconds()),

                *(propagated_state_array[ind].in_frame(frame_QSW).get_position().get_coordinates() - reference_state_array[ind].in_frame(frame_QSW).get_position().get_coordinates()),
                
                # print(f'Prop:{(propagated_state_array[ind].in_frame(frame_QSW).get_position().get_coordinates())}'),
                # print(f'Ref:{(reference_state_array[ind].in_frame(frame_QSW).get_position().get_coordinates())}'),
                *(propagated_state_array[ind].in_frame(frame_QSW).get_velocity().get_coordinates() - reference_state_array[ind].in_frame(frame_QSW).get_velocity().get_coordinates()),
            ]

In [ ]:
orbit_data_diff_QSW = [to_dataframe_QSW_diff(ind) for ind in range(0,len(propagated_state_array))]

In [ ]:
orbit_df_diff_QSW = pd.DataFrame(orbit_data_diff_QSW, columns=['$Time^{UTC}$', 'Elapsed secs', '${\delta}x_{x}^{RTN}$', '${\delta}x_{y}^{RTN}$', '${\delta}x_{z}^{RTN}$', '${\delta}v_{x}^{RTN}$', '${\delta}v_{y}^{RTN}$', '${\delta}v_{z}^{RTN}$'])

orbit_df_diff_QSW.tail()

# Validation Plots 

Plot relative position differences in RTN

In [ ]:
orbit_df_diff_QSW_position = orbit_df_diff_QSW[['Elapsed secs', '${\delta}x_{x}^{RTN}$', '${\delta}x_{y}^{RTN}$', '${\delta}x_{z}^{RTN}$']]

figure = go.Figure()
figure.update_layout(title=f'Position Difference of OSTk vs GMAT in RTN Frame {plot_title_extension}', showlegend = True, height=1000)
figure.update_xaxes(title_text="Time Elapsed (s)")
figure.update_yaxes(title_text="Position Difference in RTN (m)")

figure.add_trace(go.Scatter(
        x = orbit_df_diff_QSW_position['Elapsed secs'],
        y = orbit_df_diff_QSW_position['${\delta}x_{x}^{RTN}$'],
        name = 'Radial',
        mode = 'lines'
    ))

figure.add_trace(go.Scatter(
        x = orbit_df_diff_QSW_position['Elapsed secs'],
        y = orbit_df_diff_QSW_position['${\delta}x_{y}^{RTN}$'],
        name = 'Tangential',
        mode = 'lines'
    ))

figure.add_trace(go.Scatter(
        x = orbit_df_diff_QSW_position['Elapsed secs'],
        y = orbit_df_diff_QSW_position['${\delta}x_{z}^{RTN}$'],
        name = 'Normal',
        mode = 'lines'
    ))
    
figure.show()


Plot relative velocity differences in RTN

In [ ]:
orbit_df_diff_QSW_velocity = orbit_df_diff_QSW[['Elapsed secs', '${\delta}v_{x}^{RTN}$', '${\delta}v_{y}^{RTN}$', '${\delta}v_{z}^{RTN}$']]

figure = go.Figure()
figure.update_layout(title=f'Velocity Difference of OSTk vs GMAT in RTN Frame {plot_title_extension}', showlegend = True, height=1000)
figure.update_xaxes(title_text="Time Elapsed (s)")
figure.update_yaxes(title_text="Velocity Difference in RTN (m/s)")

figure.add_trace(go.Scatter(
        x = orbit_df_diff_QSW_velocity['Elapsed secs'],
        y = orbit_df_diff_QSW_velocity['${\delta}v_{x}^{RTN}$'],
        name = 'Radial',
        mode = 'lines'
    ))

figure.add_trace(go.Scatter(
        x = orbit_df_diff_QSW_velocity['Elapsed secs'],
        y = orbit_df_diff_QSW_velocity['${\delta}v_{y}^{RTN}$'],
        name = 'Tangential',
        mode = 'lines'
    ))

figure.add_trace(go.Scatter(
        x = orbit_df_diff_QSW_velocity['Elapsed secs'],
        y = orbit_df_diff_QSW_velocity['${\delta}v_{z}^{RTN}$'],
        name = 'Normal',
        mode = 'lines'
    ))
    
figure.show()
